# <b>AASD 4014 - Advance Mathematical Concepts for Deep Learning Group Project 1</b>

### <b>Member:</b> 1. Saksham Prakash (101410709) 2. Sik Yin Sun (101409665)

### <b>Data - Chinese Sentiment</b>
<font size=3>The link of Data:</font>
https://huggingface.co/datasets/sepidmnorozy/Chinese_sentiment


In [2]:
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from wordcloud import WordCloud
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### <b>1. Data Exploration</b>

In [3]:
dataset = load_dataset("sepidmnorozy/Chinese_sentiment")

Found cached dataset csv (C:/Users/Samuels/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Chinese_sentiment-0f90f54b0dfdaa37/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset["train"][0]

{'label': 1,
 'text': '酒店 房間 算 不錯 ， 大 床 很 舒適 。 晚上 睡的 很香 ， 酒店 環境 不到 四星 標準 ， 但 又 比 三星 高 。 地點 很好 ， 價錢 適中 。'}

In [5]:
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])
df_valid = pd.DataFrame(dataset["validation"])

In [6]:
df = pd.concat([df_train, df_test, df_valid], ignore_index=True)

In [7]:
df.head()

,label,text
0,1,酒店 房間 算 不錯 ， 大 床 很 舒適 。 晚上 睡的 很香 ， 酒店 環境 不到 四星...
1,1,"對於 這本 鯉 , 我是 針對 於 它 每期 的 主題 而 買 的 ， 第一次 買 “ 鯉 ..."
2,0,晚上 朋友 帶 我們 去 吃 海鮮 ， 結果 半夜 兩點 兩人 都 上吐下瀉 ， 請 服務 ...
3,1,教育 兩個 字 看似 簡單 ， 簡單 到 日常 的 威權 教育 。 於是 孩子 成了 父母 ...
4,1,"比較 老 的 飯店 了 , 房間 只能 用 還 比較 乾淨 整潔 來 形容 , 但 離 五星..."


In [8]:
df.shape

(19835, 2)

In [9]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [10]:
df['label'].value_counts()

1    11914
0     7921
Name: label, dtype: int64

<font size=3>After a brief exploration of the data, there is no null value in either the label or text columns. Additionally, we found out that there is more postive review than negative review.</font>

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Samuels\.cache\huggingface\datasets\sepidmnorozy___csv\sepidmnorozy--Chinese_sentiment-0f90f54b0dfdaa37\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-5cc26f370a198c0c.arrow


Map:   0%|          | 0/4896 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\Samuels\.cache\huggingface\datasets\sepidmnorozy___csv\sepidmnorozy--Chinese_sentiment-0f90f54b0dfdaa37\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-fdde51e6aa28adac.arrow
